In [1]:
import ee
import geemap

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-renjiewu660')

In [ ]:
m = geemap.Map(basemap='SATELLITE')
m

In [4]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

#MVI
def MVI(image):
    return image.addBands(image.expression('((nir-green)/(swir-green))/10',
    {'nir': image.select('B8'),
     'swir': image.select('B11'),
     'green': image.select('B3'),
    }) \
         .rename("MVI"))

# 计算 EVI
def EVI(image):
    evi = image.expression(
        "2.5 * (B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1)",
        {
            "B8": image.select("B8"),  # 近红外波段 (NIR)
            "B4": image.select("B4"),  # 红光波段 (Red)
            "B2": image.select("B2"),  # 蓝光波段 (Blue)
        },
    ).rename("EVI")
    
    return image.addBands(evi)


def LSWI(image):
    lswi = image.expression(
        "(NIR - SWIR) / (NIR + SWIR)",
        {
            "NIR": image.select("B8"),  # 近红外波段 (NIR)
            "SWIR": image.select("B11"),  # 短波红外波段 (SWIR)
        },
    ).rename("LSWI")
    
    return image.addBands(lswi)
    

def select_bands(image):
    return image.select(['B2', 'B3', 'B4','B8','EVI','LSWI','MVI'])

roi = m.user_roi
dataset = (
    ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
    .filterDate('2021-01-01', '2021-12-31')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
    .map(EVI)
    .map(LSWI)
    .map(MVI)
    .map(select_bands)
)
image = dataset.median().clip(roi)
image = image.multiply(255).add(0.5).toUint8()

# image = image.toFloat()
ele_data= ee.Image('MERIT/DEM/v1_0_3')
dem = ele_data.select('dem').clip(roi)
elevation = dem.add(0.5).toUint8()
image = image.addBands(elevation)
image.bandNames().getInfo()


['B2', 'B3', 'B4', 'B8', 'EVI', 'LSWI', 'MVI', 'dem']

In [ ]:
m.addLayer(image,{'min':0,'max':100,'bands':['B4','B3','B2']},'image')
m

In [ ]:
geemap.download_ee_image(
        image,
        region=roi,
        filename=f'test.tif',
        scale=10,
        crs="EPSG:4326",
        max_tile_size=4,
        max_tile_dim=2048
    )